
# Ch07. Sequential Modeling


## RNN

- 시간 또는 순서 정보 사용해 입력을 학습하는 것
- 현재 입력값과 직전 은닉 상태 hidden state를 참조하여 현재 상태 결정, 여러 time-step에 걸쳐 수행

### FeedForward

- 값을 앞으로 전달
- ReLU나 다른 활성화 함수 보다 tanh 주로 사용

### BPTT

- Back-Propagation Through Time; 시간 축에 대해 수행되는 역전파 방법
- t가 0에 가까워질수록 RNN 파라미터의 기울기는 각 time-step별 기울기가 더해져 점점 커지는 것
- time-step  수만큼 계층layer이 있는 것과 비슷한 속성, time-step 길어길수록 매우 깊은 신경망과 유사하게 동작

### Vanishing Gradient : 기울기 소실

- activation function인 tanh 양 끝 값을 반환하는 layer는 기울기가 0에 가까워져, 그 다음 미분값을 받는 layer는 제대로 된 미분값을 전달받을 수 없게 됨
- MLP에서 VG 문제가 쉽게 발생, 그러나 ReLU, Residual Connections 등장으로 해결

### Multi-Layer RNN

- 기본적인 RNN은 time-step 별로 동작, 
- 하나의 time-step 안에서 multi-layer RNN 만들수 있음
- 여러 RNN layers가 쌓여 하나의 RNN을 이룰 때 가장 위층의 hidden-state가 전체 RNN의 출력값
- 층별로 파라미터 값 따로 가짐, 공유X, 일반적으로 각 층 사이에 dropout 끼워 넣기도 함

### bi-directional RNN

- 기존 정방향에 역방향 추가되어 마지막 time-step에서부터 거꾸로 역방향 reverse direction으로 입력받아 진행
- 이때도 파라미터는 공유X
- 경우에 따라 전체 RNN layers 중 일부만 bi-directional RNN layer로 사용

### NLP에서 RNN 적용하는 사례

- many to one : classification ; text input - one-hot-encoding - embedding layer - dense vector - RNN - softmax - output
- one / many to many ; classify morphs ; input - one-hot-encoding - embedding layer - dense vector - RNN - time-step별 output - time-step별 softmax - multinoulli distribution
- AR model (autoregressive model) ; 입출력이 서로 같은 데이터 공유하는 경우 bi-directional RNN 사용 불가 ; morphs classification처럼 현재 출력이 다음 time-step 입력에 영향을 미치지 않는 경우는 사용 가능

## LSTM

- Long Short-Term Memory
- vanilla RNN은 time-step 길어질수록 앞의 데이터 기억하지 못하는 치명적인 단점
- 기억력 증가 : 기존 hidden-state 외에도 별도의 cell state 변수를 두고, 여러가지 gate, 망각/출력 데이터 양을 상황에 따라 효과적으로 제어
- 상대적으로 복잡한 수식, 더 많은 데이터 이용해, 더 오래 훈련

구조
- 각 gate 앞에 sigmoid를 붙여 얼마나 gate를 열고 닫을지 결정
- 결정 값에 따라 cell state 새로 encoding
- multi-layer, bi-directional 가능

## GRU

- Gated Recurrent Unit
- LSTM의 간소화 버전; 더 간단하면서, 비슷한 성능
- 성능 차이 보다는 LSTM과 GRU의 학습률, hidden size 등 hyperparameter가 다름, 연구자 관성에 따라 사용 경향 달라짐; 아직까지는 LSTM 사용 빈도가 더 높음

## Gradient Clipping

- BPTT에 따라 출력 길이가 길수록 기울기가 너무 커질 수 있음
- 학습률 조절하여 GD의 update 속도 조절
- norm (gradient size)이 너무 클 경우, learnig-rate를 아주 작게 ; 훈련 속도 매우 느려짐
- Gradient Clipping : 신경망 파라미터의 norm (일반적으로 L2 norm)을 구하고, 이 norm의 크기를 제한; gradient vector의 방향은 유지하되 크기는 학습이 망가지지 않을 정도로 축소
- 최댓값threshold은 사용자가 지정, 최댓값보다 큰 norm 가진 gradient vector에 대해서만 gradient clipping 수행 ; 학습의 발산 방지, 기울기 방향 자체는 바뀌지 않고 유지 - 학습 방향 유지, rl = 1 처럼 큰 값도 학습에 사용 가능
- 기존 SGD 방식 아닌 Adam 등 동적 학습률 optimizer 사용하는 경우 굳이 gradient clipping 적용하지 않아도 되지만, 안전장치로 적용가능






In [0]:
import torch.optim as optim
import torch.nn.utils as torch_utils


In [0]:
learning_rate = 1.
max_grad_norm = 5.

optimizer = optim.SGD(model.parameters(), lr = learning_rate)

# Apply Gradient Clipping
torch_utils.clip_grad_norm_(model.parameters(), max_grad_norm)

# Take a step of Gradient Descent
optimizer.step()